In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from rasterio.warp import reproject, calculate_default_transform, Resampling
from rasterio.features import shapes
from rasterio.features import rasterize
from rasterio.transform import from_origin
import rasterio.mask
from rasterio.io import MemoryFile
from shapely.geometry import shape
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [ ]:
tif_iec1 = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/capacity/GBR_capacity-factor_IEC1.tif"

uk = gpd.read_file(uk_boundary_path)
uk["geometry"] = uk.geometry.make_valid()
uk = uk[uk.geometry.notna() & ~uk.geometry.is_empty]
geoms = [uk.unary_union]

with rasterio.open(tif_iec1) as src:
    arr, out_transform = rasterio.mask.mask(src, geoms, crop=True, filled=True, nodata=np.nan)
    cf = arr[0].astype("float32")
    if src.nodata is not None:
        cf = np.where(cf == src.nodata, np.nan, cf)

h, w = cf.shape
left, top = out_transform * (0, 0)
right, bottom = out_transform * (w, h)
extent = (left, right, bottom, top)

vmin = float(np.nanmin(cf))
vmax = float(np.nanmax(cf))

cmap = LinearSegmentedColormap.from_list("blue_yellow", ["blue", "yellow"])
cmap.set_bad(alpha=0)

fig, ax = plt.subplots(figsize=(9, 11), dpi=600)

im = ax.imshow(cf, extent=extent, origin="upper", cmap=cmap, vmin=vmin, vmax=vmax)

uk.boundary.plot(ax=ax, color="black", linewidth=0.3)

ax.set_title("Capacity Factor (IEC1)")
ax.set_axis_off()

cbar = plt.colorbar(im, ax=ax, fraction=0.035, pad=0.02)
cbar.set_label("Capacity factor")

plt.tight_layout()
plt.savefig(out_png, dpi=600, bbox_inches="tight")
plt.show()

print("vmin/vmax:", vmin, vmax)

In [ ]:
threshold = 0.30

uk = gpd.read_file(uk_boundary_path)
uk["geometry"] = uk.geometry.make_valid()
uk = uk[uk.geometry.notna() & ~uk.geometry.is_empty]
geoms = [uk.unary_union]

with rasterio.open(tif_iec1) as src:
    arr, out_transform = rasterio.mask.mask(src, geoms, crop=True, filled=True, nodata=np.nan)
    cf = arr[0].astype("float32")
    if src.nodata is not None:
        cf = np.where(cf == src.nodata, np.nan, cf)

h, w = cf.shape
left, top = out_transform * (0, 0)
right, bottom = out_transform * (w, h)
extent = (left, right, bottom, top)

mask = np.where(np.isfinite(cf), (cf >= threshold).astype(np.uint8), np.nan)
mask = np.ma.masked_invalid(mask)

cmap = ListedColormap(["purple", "green"])
cmap.set_bad(alpha=0)

fig, ax = plt.subplots(figsize=(9, 11), dpi=600)
ax.imshow(mask, extent=extent, origin="upper", cmap=cmap, vmin=0, vmax=1)

uk.boundary.plot(ax=ax, color="black", linewidth=0.3)

ax.legend(
    handles=[
        Patch(facecolor="green", edgecolor="none", label=f"capacity ≥ {int(threshold*100)}%"),
        Patch(facecolor="purple", edgecolor="none", label=f"capacity < {int(threshold*100)}%"),
    ],
    loc="upper left",
    frameon=True
)

ax.set_axis_off()
plt.tight_layout()
plt.savefig(out_png, dpi=600, bbox_inches="tight")
plt.show()